In [3]:
import sys
sys.path.append('../../')
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from model import GTN
import pdb
import pickle
import argparse
from utils import f1_score
from scripts.data_loader import data_loader
import scipy.sparse as sp
from augment import Augmenter
from tools import evaluate_results_nc


def consis_loss(logps, temp):
    ps = [torch.exp(p) for p in logps]
    sum_p = 0.
    for p in ps:
        sum_p = sum_p + p
    avg_p = sum_p/len(ps)
    #p2 = torch.exp(logp2)
    
    sharp_p = (torch.pow(avg_p, 1./temp) / torch.sum(torch.pow(avg_p, 1./temp), dim=1, keepdim=True)).detach()
    loss = 0.
    for p in ps:
        loss += torch.mean((p-sharp_p).pow(2).sum(1))
    loss = loss/len(ps)
    return args.rate * loss


def load_data(args):
    dataset, full, feats_type = args.dataset, args.full, args.feats_type
    drop_feat = []
    if dataset == 'DBLP':
        if feats_type == 1:
            drop_feat = [0,1,3]
        elif feats_type == 2:
            drop_feat = [0]
    dl = data_loader('/ossfs/workspace/HGB/NC/benchmark/methods/GTN/'+dataset)
    if dataset == 'DBLP' and not full:
        dl.get_sub_graph([0,1,3])
    if dataset == 'ACM' and not full:
        dl.get_sub_graph([0,1,2])
    if dataset == 'ogb_4l' and not full:
        dl.get_sub_graph([0,1,2])
    edges = list(dl.links['data'].values())
    node_features = []
    w_ins = []
    for k in dl.nodes['attr']:
        th = dl.nodes['attr'][k][:,:128]
        if th is None or k in drop_feat:
            cnt = dl.nodes['count'][k]
            node_features.append(sp.eye(cnt))
            w_ins.append(cnt)
        else:
            node_features.append(th)
            w_ins.append(th.shape[1])
    val_ratio = 0.2
    train_idx = np.nonzero(dl.labels_train['mask'])[0]
    np.random.shuffle(train_idx)
    split = int(train_idx.shape[0]*val_ratio)
    val_idx = train_idx[:split]
    train_idx = train_idx[split:]
    train_idx = np.sort(train_idx)
    val_idx = np.sort(val_idx)
    test_idx = np.nonzero(dl.labels_test['mask'])[0]
    labels = np.zeros((dl.nodes['count'][0], dl.labels_train['num_classes']), dtype=int)
    labels[train_idx] = dl.labels_train['data'][train_idx]
    labels[val_idx] = dl.labels_train['data'][val_idx]
    labels = labels.argmax(axis=1)
    train_label = labels[train_idx]
    val_label = labels[val_idx]
    train_label = list(zip(train_idx.tolist(), train_label.tolist()))
    val_label = list(zip(val_idx.tolist(), val_label.tolist()))
    labels = [train_label, val_label, test_idx.tolist()]
    return node_features, edges, labels, dl, w_ins

def sp_to_spt(mat):
    coo = mat.tocoo()
    values = coo.data
    indices = np.vstack((coo.row, coo.col))

    i = torch.LongTensor(indices)
    v = torch.FloatTensor(values)
    shape = coo.shape

    return torch.sparse.FloatTensor(i, v, torch.Size(shape))

def mat2tensor(mat):
    if type(mat) is np.ndarray:
        return torch.from_numpy(mat).type(torch.FloatTensor)
    return sp_to_spt(mat)

if __name__ == '__main__':
    parser = argparse.ArgumentParser()
    parser.add_argument('--dataset', type=str, default='ogb_3l',
                        help='DATASET')
    parser.add_argument('--epoch', type=int, default=40,
                        help='Training Epochs')
    parser.add_argument('--node_dim', type=int, default=64,
                        help='Node dimension')
    parser.add_argument('--num_channels', type=int, default=2,
                        help='number of channels')
    parser.add_argument('--lr', type=float, default=0.1,
                        help='learning rate')
    parser.add_argument('--weight_decay', type=float, default=1e-3,
                        help='l2 reg')
    parser.add_argument('--num_layers', type=int, default=2,
                        help='number of layer')
    parser.add_argument('--norm', type=str, default='true',
                        help='normalization')
    parser.add_argument('--adaptive_lr', type=str, default='false',
                        help='adaptive learning rate')
    parser.add_argument('--full', type=bool, default=False)
    parser.add_argument('--feats_type', type=int, default=0)
    parser.add_argument('--device', type=int, default=0)
    parser.add_argument('--cf_k', type=int, default=2)
    parser.add_argument('--tem', type=float, default=0.2)
    parser.add_argument('--rate', type=float, default=0.2)
    args = parser.parse_args(args=[])
    print(args)
    epochs = args.epoch
    node_dim = args.node_dim
    num_channels = args.num_channels
    lr = args.lr
    weight_decay = args.weight_decay
    num_layers = args.num_layers
    norm = args.norm
    adaptive_lr = args.adaptive_lr
    
    if args.device >= 0:
        device = torch.device("cuda:"+str(args.device))
    else:
        device = torch.device('cpu')

    """with open('data/'+args.dataset+'/node_features.pkl','rb') as f:
        node_features = pickle.load(f)
    with open('data/'+args.dataset+'/edges.pkl','rb') as f:
        edges = pickle.load(f)
    with open('data/'+args.dataset+'/labels.pkl','rb') as f:
        labels = pickle.load(f)"""
    node_features, edges, labels, dl, w_ins = load_data(args)
    
    num_nodes = edges[0].shape[0]
    for i,edge in enumerate(edges):
        if i ==0:
            A = torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)
        else:
            A = torch.cat([A,torch.from_numpy(edge.todense()).type(torch.FloatTensor).unsqueeze(-1)], dim=-1)
    A = torch.cat([A,torch.eye(num_nodes).type(torch.FloatTensor).unsqueeze(-1)], dim=-1).to(device)
#     A[torch.isnan(A)]=0.01
#     print('A>>>>>>>>>',torch.any(torch.isnan(A)))
    
    node_features = [mat2tensor(x) for x in node_features] # torch.from_numpy(node_features).type(torch.FloatTensor)
#     print('node_features>>>>',type(node_features),len(node_features[1]),len(node_features[1][0]))
    train_node = torch.from_numpy(np.array(labels[0])[:,0]).type(torch.LongTensor).to(device)
    train_target = torch.from_numpy(np.array(labels[0])[:,1]).type(torch.LongTensor).to(device)
    valid_node = torch.from_numpy(np.array(labels[1])[:,0]).type(torch.LongTensor).to(device)
    valid_target = torch.from_numpy(np.array(labels[1])[:,1]).type(torch.LongTensor).to(device)
    test_node = torch.from_numpy(np.array(labels[2])).type(torch.LongTensor).to(device)
    unlabel_idx = test_node.cpu()
    
    num_classes = torch.max(train_target).item()+1
    final_f1 = 0
    for l in range(1):
        model = GTN(num_edge=A.shape[-1],
                            num_channels=num_channels,
                            w_ins = w_ins, #node_features.shape[1],
                            w_out = node_dim,
                            num_class=num_classes,
                            num_layers=num_layers,
                            norm=norm).to(device)
        if adaptive_lr == 'false':
            optimizer = torch.optim.Adam(model.parameters(), lr=args.lr, weight_decay=args.weight_decay)
        else:
            optimizer = torch.optim.Adam([{'params':model.weights},
                                        {'params':model.linear1.parameters()},
                                        {'params':model.linear2.parameters()},
                                        {"params":model.layers.parameters(), "lr":0.05}
                                        ], lr=0.005, weight_decay=args.weight_decay)
        loss = nn.CrossEntropyLoss().to(device)
        # Train & Valid & Test
        best_val_loss = 10000
        best_test_loss = 10000
        best_train_loss = 10000
        best_train_f1 = 0
        best_val_f1 = 0
        best_test_f1 = 0
        test_embeddings = []
        
        augmenter = Augmenter(args=args, features_list=node_features, is_label=labels)
        
    
        for i in range(epochs):
            loss_u_list = []
            
            for param_group in optimizer.param_groups:
                if param_group['lr'] > 0.005:
                    param_group['lr'] = param_group['lr'] * 0.9
            print('Epoch:  ',i+1)
            model.zero_grad()
            model.train()
            
            # 正向传播时：开启自动求导的异常侦测
#             torch.autograd.set_detect_anomaly(True)
            
            loss_l,y_train,Ws,embed,all_y = model(A, node_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y[unlabel_idx], dim=-1))
            # ------ 节点增强 ------
            aug_features = augmenter.node_aug_heterograph_ogbm(node_features, unlabel_idx, args.cf_k)
            _,y_train_u1,Ws_u1,embed1,all_y1 = model(A, aug_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y1[unlabel_idx], dim=-1))
#             # ------ 结构增强 ------
            aug_A = augmenter.triangle_aug_heterograph()
            _,y_train_u2,Ws_u2,embed2,all_y2 = model(aug_A, node_features, train_node, train_target)
            loss_u_list.append(torch.log_softmax(all_y2[unlabel_idx], dim=-1))
            
            loss_u = consis_loss(loss_u_list, temp=args.tem)
            loss = loss_l + loss_u
            
            train_f1 = torch.mean(f1_score(torch.argmax(y_train.detach(),dim=1), train_target, num_classes=num_classes)).cpu().numpy()
            print('Train - Loss: {}, Macro_F1: {}'.format(loss.detach().cpu().numpy(), train_f1))
            
#             for name, parms in model.named_parameters():
#                 print('-->name:', name, '-->grad_requirs:', parms.requires_grad)
#                 print('--weight', torch.mean(parms.data))
#                 print('-->grad_value:', torch.mean(parms.grad)) 
            # 反向传播时：在求导时开启侦测
#             with torch.autograd.detect_anomaly():
            loss.backward()
    
            optimizer.step()
            model.eval()
            # Valid
            with torch.no_grad():
#                 print('A 2 2>>>>>>>>>',torch.any(torch.isnan(A)))
                val_loss,y_valid,_,_,all_y_valid = model(A, node_features, valid_node, valid_target)
                val_f1 = torch.mean(f1_score(torch.argmax(y_valid,dim=1), valid_target, num_classes=num_classes)).cpu().numpy()
                print('Valid - Loss: {}, Macro_F1: {}'.format(val_loss.detach().cpu().numpy(), val_f1))
                test_loss,y_test,W,test_emb,_ = model(A, node_features, test_node, None)
                pred = y_test.cpu().numpy().argmax(axis=1)
                onehot = np.eye(num_classes, dtype=np.int32)
                pred = onehot[pred]
                test_f1 = dl.evaluate(pred)
                print('test_f1>>>>>> ',test_f1)
                
                #test_f1 = torch.mean(f1_score(torch.argmax(y_test,dim=1), test_target, num_classes=num_classes)).cpu().numpy()
                #print('Test - Loss: {}, Macro_F1: {}\n'.format(test_loss.detach().cpu().numpy(), test_f1))
            if val_f1 > best_val_f1:
                best_val_loss = val_loss.detach().cpu().numpy()
                #best_test_loss = test_loss.detach().cpu().numpy()
                best_train_loss = loss.detach().cpu().numpy()
                best_train_f1 = train_f1
                best_val_f1 = val_f1
                best_test_f1 = test_f1 
                test_embeddings = test_emb
        print('---------------Best Results--------------------')
        print('Train - Loss: {}, Macro_F1: {}'.format(best_train_loss, best_train_f1))
        print('Valid - Loss: {}, Macro_F1: {}'.format(best_val_loss, best_val_f1))
        print('best_test_f1>>>>>> ',best_test_f1)
        svm_macro_f1_list, svm_micro_f1_list, nmi_mean, nmi_std, ari_mean, ari_std = evaluate_results_nc(
                test_embeddings[test_node].cpu().numpy(), dl.labels_test['data'][dl.labels_test['mask']].argmax(axis=1), num_classes=dl.labels_test['num_classes'])

#         print('Test - Loss: {}, Macro_F1: {}'.format(best_test_loss, best_test_f1))
#         final_f1 += best_test_f1


Namespace(adaptive_lr='false', cf_k=2, dataset='ogb_4l', device=0, epoch=40, feats_type=0, full=False, lr=0.1, node_dim=64, norm='true', num_channels=2, num_layers=2, rate=0.2, tem=0.2, weight_decay=0.001)
loading data:
loading data >>>>>>>>>>>>>>>>>>>>>
0 {'0': 3060}
1 {'0': 3060, '1': 7965}
2 {'0': 3060, '1': 7965, '2': 6934}
3 {'0': 3060, '1': 7965, '2': 6934, '3': 3990}
building dgl.heterograph >>>>>>>>>>>>>>>>>>>>>
sample_num: 56323
Epoch:   1
node_features>>>>>>  dict_items([(0, (1, 2)), (1, (1, 0)), (2, (0, 0)), (3, (0, 3)), (4, (2, 1)), (5, (0, 1)), (6, (0, 0)), (7, (3, 0))])


IndexError: index 16604 is out of bounds for dimension 0 with size 11025

In [ ]:
1